In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

In [2]:
df = pd.read_csv("sp_500_stocks.csv")
df.head()

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV


In [11]:
# saving the token for authentication when using API
# saving the base url in a separate variable
# making a list of column names for use in the dataframe
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'
base_url = "https://sandbox.iexapis.com/stable/stock/"
col_names = ["Ticker", "Stock Price", "Market Capitalization"]

In [12]:
#here we define a function that divides the list into 100 sized chunks
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

In [18]:
# we first form 100 sized chunks for batch_api call
groups = list(chunks(df["Ticker"], 100))

symbol_strings = []

final_dataframe = pd.DataFrame(columns = col_names)

for g in groups:
    symbol_strings.append(','.join(g))

for symbol_string in symbol_strings:
    batch_api_call_url = f"{base_url}market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        if symbol == "WLTW":
            continue
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]["quote"]["latestPrice"],
                    data[symbol]["quote"]["marketCap"]
                ],
                index = col_names
            ),
            ignore_index = True
        )

final_dataframe

,Ticker,Stock Price,Market Capitalization
0,A,139.570,41196702332
1,AAL,18.960,12566411910
2,AAP,224.630,14113620002
3,AAPL,175.400,2900436209862
4,ABBV,147.056,263733839205
...,...,...,...
499,YUM,127.330,36424962040
500,ZBH,125.030,25887130323
501,ZBRA,447.400,24296260047
502,ZION,76.910,11693331437


In [36]:
# we take the quantity of money that the trades likes to invest in S&P 500 stocks
print("Please enter your portfolio size :")
portfolio_size = input()

try:
    portfolio_size = int(portfolio_size)
except ValueError:
    print("Value entered is not a number!\n Enter again :")
    portfolio_size = input()
    portfolio_size = int(portfolio_size)

Please enter your portfolio size :
10000000


In [37]:
# we then calculate what will be the money for each stock
position_size = portfolio_size/final_dataframe.shape[0]

In [46]:
# we now enter the stocks that the trader needs to buy into the final_dataframe
a = np.asarray(final_dataframe.loc[:, "Stock Price"])
a = np.floor(position_size*(1/a))

final_dataframe.loc[:, "Number of shares to buy"] = a

In [50]:
#creating the writer object for writing to excel files
writer = pd.ExcelWriter("recommended trades.xlsx", engine="xlsxwriter")
final_dataframe.to_excel(writer, "Recommended Trades", index = False)

In [55]:
# creating different formats for strings, numbers and monetary values
background_color = "0a0a23"
font_color = "#ffffff"

string_format = writer.book.add_format(
    {
        "font_color": font_color,
        "bg_color": background_color,
        "border": 1
    }
)

dollar_format = writer.book.add_format(
    {
        "num_format": "$0.00",
        "font_color": font_color,
        "bg_color": background_color,
        "border": 1
    }
)

number_format = writer.book.add_format(
    {
        "num_format": "0",
        "font_color": font_color,
        "bg_color": background_color,
        "border": 1
    }
)

In [56]:
# incorporating the various formats into the sheet and then finally saving it
column_formats = {
    "A" : ["Ticker", string_format],
    "B" : ["Latest Price", dollar_format],
    "C" : ["Market Capitalization", number_format],
    "D" : ["Number of shares to buy", number_format]
}

for column in column_formats.keys():
    writer.sheets["Recommended Trades"].set_column(f"{column}:{column}", 18, column_formats[column][1])
    writer.sheets["Recommended Trades"].write(f"{column}1", column_formats[column][0], column_formats[column][1])

writer.save()